# Merge the fine-tuned model

To try your fine-tuned Llama2 model, you have to merge it with the foundational base model version used for the fine-tuning.

`Make sure to install the required dependancies first, preferable in Conda environment with PyTorch 2.0+`

Before you start make sure that you've installed the `pytorch20_p39_gpu_v2` Conda and activate it in the `Terminal`

```bash
odsc conda install -s pytorch20_p39_gpu_v2
```

... then activate it

```bash
conda activate /home/datascience/conda/pytorch20_p39_gpu_v2
```

Then continue by starting the notebook and selecting the `pytorch20_p39_gpu_v2` as Python Kernel, then install the dependancies...

In [ ]:
!pip install tokenizers==0.13.3 -U && pip install transformers -U

In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
from peft.utils import PeftConfig
import torch
    
import os

Set the variables to point to:

`ft_model` - the location of the object storage mount to your OCI Data Science Notebook where the PEFT weights where stored
`output_dir` - where to store the merged weights, preferably again on object storage
`base_model_name_or_path` - name of the model card as available in Hugging Face
`your_huggingface_auth_token` - your Hugging Face auth token

In [ ]:
ft_model = "/mnt/llama2/outputs/13b-hf/ocid1.datasciencejob.oc1.eu-frankfurt-1.amaaaaaan"

output_dir = "/mnt/llama2/llama2-ft-lora-13b-merged"

base_model_name_or_path = "meta-llama/Llama-2-13b-hf"

your_huggingface_auth_token = "<your-huggingface-login-token>"

os.makedirs(output_dir, exist_ok=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path, use_auth_token=your_huggingface_auth_token)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    device_map="auto", 
    return_dict=True,
    torch_dtype=torch.bfloat16,
    use_auth_token=your_huggingface_auth_token
)

In [ ]:
model = PeftModel.from_pretrained(base_model, ft_model)

In [ ]:
print(f"\nModel architecture before merging", flush=True)
print(model)

In [ ]:
model = model.merge_and_unload()

print(f"\nModel architecture after merging", flush=True)
print(model)

In [ ]:
ppl = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    use_cache=True,
    device_map="auto"
)

In [ ]:
dialog = """Martin: I won two cinema tickets! Aggie: oh cool, how come? Martin: online. on fb, the movie mag organized it Aggie: so what did you do Martin: just write a short review and that's it Aggie: well done :) so what and when. and where? Martin: the new film with Redford Aggie: i guess i heard sth Martin: it's pretty cool i heard. till the end of the week Aggie: sounds good. we'll find time XD"""


In [ ]:
print(ppl(f"Summarize this dialog:\n{dialog}\n---",
    max_length=200,
        temperature=0.95,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id)[0]['generated_text'])

In [ ]:
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Export to Model Catalog

In [ ]:
import shutil
from ads.model.datascience_model import DataScienceModel
from ads.common.auth import default_signer
import ads
import os

In [ ]:
ads.set_auth("resource_principal")

In [ ]:
MODEL_PATH=output_dir
ARTFICAT_FILE_NAME=f"{os.path.basename(output_dir)}.zip"

In [ ]:
os.path.isfile(ARTFICAT_FILE_NAME)

In [ ]:
# model_archive_file = shutil.make_archive(ARTFICAT_FILE_NAME, format="zip", root_dir=MODEL_PATH)
!zip -0 -rj $ARTFICAT_FILE_NAME $MODEL_PATH/*

In [ ]:
os.path.isfile(ARTFICAT_FILE_NAME)

In [ ]:
MODEL_DISPLAY_NAME = "LLama2-FT-Lora-13b-Samsum"
bucket = "your bucket"
namespace = "your namespace"
OCI_BUCKET = f"oci://{bucket}@{namespace}/datascience-large-artifact-store/" # For large models we first upload the model to object storag and then provide the object storage location while creating model artifact

In [ ]:
model = DataScienceModel()
model = (
    model.with_display_name(MODEL_DISPLAY_NAME)
    .with_artifact(ARTFICAT_FILE_NAME)
)
model.create(bucket_uri=OCI_BUCKET)
